In [1]:
from dotenv import load_dotenv
load_dotenv()

False

In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

model = ChatOpenAI(
    model="gpt-4o-mini",   # usa un modelo que tengas habilitado
    temperature=0
)
print("🤖 Modelo listo")

🤖 Modelo listo


In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
import os

In [4]:
chat_prompt_template = ChatPromptTemplate.from_messages([
  ("system", """Eres un desarrollador sénior experto en C# y ASP.NET Web Forms (.NET Framework 4.8).
Necesito generar el esqueleto de un **Sistema de Punto de Venta (POS)** minimal pero funcional.
Usa **SQL Server** y **ADO.NET**. No usar ASP.NET Core, ni MVC, ni Razor. Solo **páginas .aspx + code-behind .cs**.
Mantén **seguridad básica** y usa **variables de sesión (Session)**, no cookies.

Directrices generales:
1) Lenguaje y estilo
   - C# idiomático para .NET Framework 4.8.
   - Convenciones: Clases/Interfaces/Métodos → PascalCase; variables/parámetros → camelCase; constantes → ALL_CAPS.
   - Comentarios XML (///) en clases y métodos públicos.

2) Acceso a datos (SQL Server) con ADO.NET
   - Usar SqlConnection, SqlCommand, SqlDataReader y bloques using.
   - **Siempre** consultas parametrizadas (NO concatenar SQL).
   - Transacciones cuando corresponda (por ejemplo, ventas).
   - Mapear columnas explícitamente y validar nulos.

3) Gestión de sesión (NO cookies)
   - Mantener datos de sesión del usuario (ej. Session["UserId"], Session["Rol"]).
   - Validar en cada página protegida si la sesión está activa antes de permitir acceso.
   - Limpiar la sesión en el logout.
   - Configurar timeout de sesión en web.config.

4) Seguridad mínima obligatoria
   - Contraseñas: **hash + salt** con Rfc2898DeriveBytes (PBKDF2). Nunca guardar texto plano.
   - Validar entrada del usuario en servidor (longitud, formato). Mantener validateRequest="true".
   - Encodar salida cuando aplique (evitar XSS).
   - SQL Injection: **parámetros en todos los comandos** y tipos correctos.
   - Manejo de errores: capturar excepciones, no mostrar detalles sensibles; log técnico.
   - Principio de mínimo privilegio en la cadena de conexión; secretos en web.config (no en código).

Arquitectura mínima por capas:
- Interfaces de repositorio (p. ej., IUsuarioRepository, IProductoRepository).
- Implementaciones ADO.NET (UsuarioRepository, ProductoRepository).
- Servicios de negocio simples (si es necesario).
- Páginas Web Forms (.aspx + .cs) solo para UI y eventos.

Contexto funcional del POS:
- **Roles**:
  - Administrador: gestiona usuarios, productos y reportes.
  - Cajero: realiza ventas y consulta reportes.
- **Módulos (UI mínima, funcional)**:
  1) Login.aspx (autenticación contra SQL Server con hash seguro)
  2) Default.aspx (pantalla principal con menú a módulos)
  3) Users.aspx (ABM de usuarios con rol)
  4) Products.aspx (ABM de productos: Nombre, SKU, Precio, Existencia, Activo)
  5) CashRegister.aspx (caja: buscar producto, capturar cantidad, calcular subtotal, IVA y total; descontar stock)
  6) SalesReport.aspx (reporte de ventas por rango de fechas)

Entregables esperados:
- Archivos .aspx con controles mínimos y validadores básicos.
- Code-behind .cs con handlers preparados (TODOs claros para conectar repos/servicios).
- Interfaces y repositorios ADO.NET con ejemplos de SELECT/INSERT/UPDATE/DELETE **parametrizados**.
- Utilidad de hash seguro para contraseñas (PBKDF2).
- Ejemplo de manejo de sesión con Session["UserId"] y validación en páginas protegidas.
- web.config de ejemplo con:
  - <connectionStrings> para SQL Server
  - <sessionState timeout="20" />
  - <pages validateRequest="true" viewStateEncryptionMode="Always" />

Formato de respuesta:
- Devuelve **solo código listo para compilar**, en bloques ```aspx y ```csharp según corresponda.
- Incluye TODOs donde falte lógica concreta.
- Agrega al final un comentario breve sobre decisiones de seguridad (parametrización, hash, validación, manejo de sesión).
"""
                ),
  ("human", "Implementa la siguiente especificación en C#. \n\n{text}")
])

In [5]:
result = chat_prompt_template.invoke({
  "text": """
Tarea:
Genera un documento maestro, en español claro y ordenado, para crear un Sistema de Punto de Venta (POS) en
**C# y ASP.NET Web Forms (.NET Framework 4.8)** usando **ADO.NET (SqlClient)** y autenticación con **Session**.
El documento debe contener 4 secciones en este orden y cada una debe construir sobre la anterior:

------------------------------------------------------
1) CONFIGURACIÓN DEL ENTORNO DE DESARROLLO
------------------------------------------------------
- Incluir solo los pasos necesarios para preparar el entorno:
  • Instalación: Visual Studio 2022 (workload ASP.NET and web development), .NET Framework 4.8, SQL Server (Express/LocalDB),
    paquete NuGet BCrypt.Net-Next (para hash seguro de contraseñas).
  • Creación del proyecto en Visual Studio:
      - Tipo: Aplicación web de ASP.NET (.NET Framework)
      - Framework: .NET 4.8
      - Plantilla: Web Forms
      - Autenticación: Sin autenticación
  • Configuración básica de Web.config:
      - connectionStrings (placeholder seguro para POS_DB)
      - globalization (es-MX)
      - compilation targetFramework="4.8"
      - Nota de usar HTTPS en desarrollo/producción.
  • **Base de datos**:
      - Describir tablas principales: Users, Products, Sales, SaleItems (campos básicos y relaciones).
      - **Al final de esta sección, incluir un SCRIPT SQL COMPLETO y EJECUTABLE** para SQL Server que:
        1) Cree la base `POS_DB`
        2) Cree las tablas `Users`, `Products`, `Sales`, `SaleItems` con claves foráneas e índices únicos (Email, Sku)
        3) Inserte un usuario Admin inicial (mostrar solo el hash BCrypt, NO la contraseña en texto plano)
  • Importante: En esta sección NO incluir estructura de carpetas ni código del backend.

------------------------------------------------------
2) ESTRUCTURA INICIAL DEL PROYECTO
------------------------------------------------------
- Basarse en el entorno definido en el punto 1.
- Presentar un **árbol de carpetas y archivos**, por ejemplo:
  /App_Code
    /Models
    /Data
    /Services
  /Pages
    Login.aspx, Default.aspx, Users.aspx, Products.aspx, CashRegister.aspx, SalesReport.aspx
  /Styles/Site.css
  /App_Themes/PosTheme/PosTheme.skin (opcional)
  Site.Master
- Describir brevemente la función de cada carpeta/archivo.
- **No incluir código**, solo organización y propósito.

------------------------------------------------------
3) BACKEND CORE MÍNIMO Y SEGURO (ADO.NET + Session)
------------------------------------------------------
- Basarse en la estructura del punto 2. Mantenerlo simple y seguro (SIN Entity Framework).
- Incluir:
  • Modelos (POCOs) en App_Code/Models:
      - User: Id, Email, PasswordHash (BCrypt), Role (Admin|Cashier), Active
      - Product: Id, Sku, Name, Price, Stock, Active
      - Sale: Id, CreatedAtUtc, CashierUserId, Subtotal, Tax, Total
      - SaleItem: Id, SaleId, ProductId, Quantity, UnitPrice, LineTotal
  • Acceso a datos (ADO.NET) en App_Code/Data:
      - Db.cs → GetConnection() leyendo POS_DB de Web.config
      - UserData, ProductData, SalesData → CRUD y consultas **con SQL parametrizado**
      - SalesData → InsertSale con **SqlTransaction** (inserta venta + items y descuenta stock)
  • Servicios en App_Code/Services:
      - AuthService → Login (verifica BCrypt; si ok setea Session["uid"] y Session["role"]), Logout (limpia Session)
      - SalesService → CreateSale (valida stock, calcula IVA 16 %, registra venta con transacción)
  • Pautas para páginas (sin código extenso):
      - En Page_Load de páginas protegidas: si Session["uid"] == null → redirigir a Login
      - Users.aspx y Products.aspx: solo acceso Admin
  • Seguridad mínima: SQL parametrizado; BCrypt; validación en servidor; anti-XSS con Server.HtmlEncode; mensajes de error genéricos.
  • En esta sección solo incluir fragmentos mínimos si son indispensables; evitar código largo.

------------------------------------------------------
4) FRONTEND (WEB FORMS CON CONTROLES ASP.NET + ESTILOS PROPIOS)
------------------------------------------------------
- Basarse en la estructura y backend previos (puntos 2 y 3).
- Usar **controles de ASP.NET** y validadores nativos.

- **Menú de navegación (header superior)**
    • Diseñar el menú en la parte superior (barra horizontal fija/sencilla).
    • Fondo del menú: `#353A40`.
    • Texto de los ítems del menú: **blanco**.
    • Mostrar opciones: Home, Users, Products, CashRegister, SalesReport, Logout.
    • Ocultar o desactivar enlaces según el rol (`Session["role"]`).
    • El diseño puede ser simple, pero bien acomodado y consistente.

- **Paleta de colores obligatoria en toda la app**
    • Menú superior (navbar): fondo `#353A40`, **texto blanco**.
    • Fondos generales: `#F5F6FA`, **texto negro**.
    • Encabezado de GridView: fondo `#19A1B9`, **texto blanco**.
    • Botones principales: fondo `#0F6AF6`, **texto blanco**.
    • Botones de acción crítica (eliminar/cancelar): fondo `#E13C4A`, **texto blanco**.
    • Todo el contenido normal (labels, texto en formularios): **negro**.

- **Site.Master**
    • Incluir el menú superior con controles ASP.NET (p. ej., `asp:Menu` o un `Repeater`) y `ContentPlaceHolder`.
    • Referenciar `Styles/Site.css` y, si se desea, `App_Themes/PosTheme`.
    • Visibilidad de opciones según `Session["role"]`.

- **Páginas (.aspx) con controles ASP.NET y validadores**
    • Login.aspx: `TextBox` Email/Password, `Button` Iniciar; `RequiredFieldValidator`, `RegularExpressionValidator`, `ValidationSummary`.
    • Default.aspx: bienvenida y rol actual.
    • Users.aspx (solo Admin): `GridView` + `FormView`/`DetailsView` para CRUD de usuarios (hash con BCrypt en backend).
    • Products.aspx (solo Admin): `GridView` + `FormView` con `RangeValidator` para Price/Stock.
    • CashRegister.aspx (Admin/Cashier): `DropDownList` productos activos, `TextBox` cantidad, botón Agregar; `GridView` carrito; Labels Subtotal/IVA/Total;
      botón Registrar venta (usa SalesService). Nota: prevenir doble submit (deshabilitar botón mientras procesa).
    • SalesReport.aspx (Admin/Cashier): filtros FechaDesde/FechaHasta con validadores; `GridView` resultados y total general.

- **Estilos**
    • Definir en `Styles/Site.css` las clases para navbar, botones y GridView, respetando la paleta de colores indicada.
    • Sencillo, limpio y con buen contraste (accesible).
    • (Opcional) `App_Themes/PosTheme/PosTheme.skin` con skins para `GridView`, `Button`, `TextBox` y `Label`.

    ------------------------------------------------------
------------------------------------------------------
5) GENERACIÓN DEL CÓDIGO DE PÁGINAS (CODE-BEHIND .ASPX.CS)
------------------------------------------------------
Objetivo:
Entregar ÚNICAMENTE el código C# de los archivos **code-behind** de las páginas Web Forms. 
No incluir markup .aspx, CSS ni clases de Data/Services aquí. 
El código debe compilar con .NET Framework 4.8 y usar ADO.NET + Session, según se definió en las secciones 1–4.

Formato de salida:
- Un bloque por archivo con el encabezado:  // === Pages/<Nombre>.aspx.cs ===
- Luego el **código C# completo** del code-behind (using, namespace, clase parcial, métodos).
- Comentarios breves en español.
- Manejo de errores con try/catch y mensajes amigables (sin detalles sensibles).

Convenciones y supuestos (comunes a todas las páginas):
- Verificar sesión al comienzo de `Page_Load`:
  if (Session["uid"] == null) { Response.Redirect("Login.aspx"); return; }
- Donde aplique, verificar rol:
  - Admin requerido en **Users.aspx** y **Products.aspx**.
- Usar `Page.IsPostBack` para separar carga inicial.
- Mostrar mensajes en un `Label` llamado **lblMessage** (si aplica).
- Nombres de servicios y métodos (ya definidos en el backend):
  - `AuthService.Login(email, pwd, Session)` → bool
  - `AuthService.Logout(Session)` → void
  - `UserData.GetAll()`, `UserData.Insert(...)`, `UserData.Update(...)`, `UserData.Delete(id)`
  - `ProductData.GetAll()`, `ProductData.Insert/Update/Delete(...)`
  - `SalesService.CreateSale(cashierUserId, IEnumerable<(int productId, int qty)>)` → int (SaleId)
  - `SalesData.GetByDateRange(DateTime fromUtc, DateTime toUtc)`
- Fechas en reportes: tratar como UTC o convertir a UTC si vienen en hora local.
- SQL siempre parametrizado (se asume dentro de Data/Services).
- Anti-XSS: al escribir cadenas ingresadas por usuario, usar `Server.HtmlEncode(...)` si corresponde.

Archivos a generar (solo code-behind):

A) // === Pages/Login.aspx.cs ===
- Controles esperados: `txtEmail`, `txtPassword`, `btnLogin`, `lblMessage`.
- Lógica:
  - `Page_Load`: si ya hay sesión → `Response.Redirect("Default.aspx")`.
  - `btnLogin_Click`: validar `Page.IsValid`, llamar `AuthService.Login(...)`. 
    Si ok → `Response.Redirect("Default.aspx")`; si falla → mensaje en `lblMessage`.
- Limpieza segura del mensaje en cada carga.

B) // === Pages/Default.aspx.cs ===
- Controles esperados: `lblWelcome`, `lblRole`.
- Lógica:
  - `Page_Load`: validar sesión, cargar nombre o email del usuario si se dispone (`UserData.GetById(...)` opcional). 
    Mostrar rol desde `Session["role"]`.

C) // === Pages/Users.aspx.cs ===  (solo Admin)
- Controles esperados:
  - `gvUsers` (GridView), `fvUser` (FormView o DetailsView), `lblMessage`.
  - Botones: `btnNew`, `btnSave`, `btnDelete` (o comandos en GridView).
- Lógica:
  - `Page_Load`: validar sesión y rol Admin; en `!IsPostBack` → `BindGrid()`.
  - `BindGrid()`: `gvUsers.DataSource = UserData.GetAll(); gvUsers.DataBind();`
  - Altas/ediciones:
    - Tomar valores de controles del FormView (email, role, active, password opcional).
    - Para password nueva: hashear con BCrypt antes de guardar.
  - Eliminación: confirmar por CommandArgument (id).
  - Manejar eventos típicos: `gvUsers_RowCommand`, `gvUsers_PageIndexChanging`, `gvUsers_RowDataBound` (si aplica).
  - Mensajes en `lblMessage`.

D) // === Pages/Products.aspx.cs ===  (solo Admin)
- Controles esperados:
  - `gvProducts`, `fvProduct`, `lblMessage`.
- Lógica:
  - `Page_Load`: validar sesión y rol Admin; en `!IsPostBack` → `BindGrid()`.
  - `BindGrid()`: cargar todos los productos.
  - Crear/editar: validar rango de `Price` y `Stock` (también hay validadores en .aspx).
  - Eliminar: por CommandArgument (id).
  - Eventos típicos: `gvProducts_RowCommand`, `gvProducts_PageIndexChanging`.

E) // === Pages/CashRegister.aspx.cs ===  (Admin/Cashier)
- Controles esperados:
  - `ddlProducts` (DropDownList de productos activos), `txtQty`, `btnAddItem`.
  - `gvCart` (GridView del carrito en memoria), `lblSubtotal`, `lblTax`, `lblTotal`, `btnCheckout`, `lblMessage`.
- Lógica:
  - `Page_Load`: validar sesión; opcional validar rol en {Admin, Cashier}.
  - Cargar `ddlProducts` en `!IsPostBack` con productos activos.
  - Carrito en memoria: usar `List<CartItem>` en `ViewState` o `Session`.
    - `CartItem` (productId, name, unitPrice, qty, lineTotal).
  - `btnAddItem_Click`: validar cantidad > 0; agregar/actualizar item; recalcular totales (IVA = 16%).
  - `RecalcTotals()`: Subtotal, Tax, Total.
  - `gvCart_RowCommand`: eliminar item.
  - `btnCheckout_Click`: construir `IEnumerable<(int productId, int qty)>` a partir del carrito y llamar `SalesService.CreateSale(...)`.
    - Si ok: limpiar carrito, mostrar SaleId en `lblMessage`.

F) // === Pages/SalesReport.aspx.cs ===  (Admin/Cashier)
- Controles esperados: `txtFrom`, `txtTo`, `btnFilter`, `gvSales`, `lblTotalGeneral`, `lblMessage`.
- Lógica:
  - `Page_Load`: validar sesión; opcional validar rol en {Admin, Cashier}.
  - `btnFilter_Click`: parsear fechas (local → UTC si aplica), pedir `SalesData.GetByDateRange(fromUtc, toUtc)`.
  - Bindear `gvSales`; calcular y mostrar total general (suma de `Total`).

G) // === Site.Master.cs ===  (manejo de logout y visibilidad simple)
- Controles esperados: botones/links `lnkLogout` y contenedores de menú (p. ej., `pnlAdmin`).
- Lógica:
  - `Page_Load`: si hay sesión, mostrar/ocultar menús por rol (`Session["role"]`).
  - `lnkLogout_Click`: `AuthService.Logout(Session); Response.Redirect("~/Pages/Login.aspx");`

Consideraciones extra:
- Usar `int.Parse`/`decimal.Parse` con cuidado; preferir `int.TryParse`/`decimal.TryParse` con mensajes claros.
- En reportes, cuando no hay datos, `gvSales.EmptyDataText = "Sin resultados en el rango seleccionado";`
- Evitar doble envío en `btnCheckout`: deshabilitar el botón y re-habilitar si falla.

Entrega:
- Generar el **código C# completo** para cada archivo listado (A–G), en ese orden, con los `using` necesarios:
  `using System; using System.Data; using System.Web; using System.Web.UI; using System.Web.UI.WebControls;`
  y los `using` de tus namespaces (`App_Code.Data`, `App_Code.Services`, etc.) según lo definido en la sección 3.

     ------------------------------------------------------

------------------------------------------------------
FORMATO DE SALIDA
------------------------------------------------------
- Redactar en español, con encabezados claros por sección.
- No repetir contenido entre secciones.
- Incluir **el script SQL completo** en la Sección 1 al final.
- En las secciones 3 y 4, incluir solo fragmentos mínimos cuando sea indispensable (evitar código extenso).
- Respetar la paleta de colores y el menú superior sencillo pero bien acomodado.
- Mantener todo claro, simple y seguro.

  """
})


In [6]:
from pathlib import Path

base = Path().resolve()
outdir = base / "results"          # <— plural
outdir.mkdir(parents=True, exist_ok=True)

outfile = outdir / "EspecificacionesProyecto.md"
texto = getattr(result, "content", None) or str(result)
outfile.write_text(texto, encoding="utf-8")

print("✅ Guardado en:", outfile.resolve())
print("✅ Guardado en:", outfile.resolve())
print("📏 Tamaño (bytes):", outfile.stat().st_size)
print("🧪 Existe?:", outfile.exists())

✅ Guardado en: /home/runner/work/PuntoVentas/PuntoVentas/results/EspecificacionesProyecto.md
✅ Guardado en: /home/runner/work/PuntoVentas/PuntoVentas/results/EspecificacionesProyecto.md
📏 Tamaño (bytes): 17603
🧪 Existe?: True


In [7]:
result2 = chat_prompt_template.invoke({
  "text": """
En Base ha esta base de datos:

- Crear la base de datos
CREATE DATABASE POS_DB;
GO

USE POS_DB;
GO

-- Crear tabla Users
CREATE TABLE Users (
    Id INT PRIMARY KEY IDENTITY(1,1),
    Email NVARCHAR(255) UNIQUE NOT NULL,
    PasswordHash NVARCHAR(255) NOT NULL,
    Role NVARCHAR(50) NOT NULL,
    Active BIT NOT NULL
);

-- Crear tabla Products
CREATE TABLE Products (
    Id INT PRIMARY KEY IDENTITY(1,1),
    Sku NVARCHAR(50) UNIQUE NOT NULL,
    Name NVARCHAR(255) NOT NULL,
    Price DECIMAL(18, 2) NOT NULL CHECK (Price >= 0),
    Stock INT NOT NULL CHECK (Stock >= 0),
    Active BIT NOT NULL
);

-- Crear tabla Sales
CREATE TABLE Sales (
    Id INT PRIMARY KEY IDENTITY(1,1),
    CreatedAtUtc DATETIME NOT NULL DEFAULT GETUTCDATE(),
    CashierUserId INT NOT NULL,
    Subtotal DECIMAL(18, 2) NOT NULL,
    Tax DECIMAL(18, 2) NOT NULL,
    Total DECIMAL(18, 2) NOT NULL,
    FOREIGN KEY (CashierUserId) REFERENCES Users(Id)
);

-- Crear tabla SaleItems
CREATE TABLE SaleItems (
    Id INT PRIMARY KEY IDENTITY(1,1),
    SaleId INT NOT NULL,
    ProductId INT NOT NULL,
    Quantity INT NOT NULL CHECK (Quantity > 0),
    UnitPrice DECIMAL(18, 2) NOT NULL,
    LineTotal DECIMAL(18, 2) NOT NULL,
    FOREIGN KEY (SaleId) REFERENCES Sales(Id),
    FOREIGN KEY (ProductId) REFERENCES Products(Id)
);

y ha estas paginas 

// === App_Code/Models/User.cs ===
public class User
{
    public int Id { get; set; }
    public string Email { get; set; }
    public string PasswordHash { get; set; }
    public string Role { get; set; }
    public bool Active { get; set; }
}

// === App_Code/Models/Product.cs ===
public class Product
{
    public int Id { get; set; }
    public string Sku { get; set; }
    public string Name { get; set; }
    public decimal Price { get; set; }
    public int Stock { get; set; }
    public bool Active { get; set; }
}

// === App_Code/Models/Sale.cs ===
public class Sale
{
    public int Id { get; set; }
    public DateTime CreatedAtUtc { get; set; }
    public int CashierUserId { get; set; }
    public decimal Subtotal { get; set; }
    public decimal Tax { get; set; }
    public decimal Total { get; set; }
}

// === App_Code/Models/SaleItem.cs ===
public class SaleItem
{
    public int Id { get; set; }
    public int SaleId { get; set; }
    public int ProductId { get; set; }
    public int Quantity { get; set; }
    public decimal UnitPrice { get; set; }
    public decimal LineTotal { get; set; }
}
Acceso a datos (ADO.NET) en App_Code/Data
// === App_Code/Data/Db.cs ===
public static class Db
{
    public static SqlConnection GetConnection()
    {
        return new SqlConnection(ConfigurationManager.ConnectionStrings["POS_DB"].ConnectionString);
    }
}

// === App_Code/Data/UserData.cs ===
public class UserData
{
    public User GetById(int id)
    {
        using (var conn = Db.GetConnection())
        {
            conn.Open();
            using (var cmd = new SqlCommand("SELECT * FROM Users WHERE Id = @Id", conn))
            {
                cmd.Parameters.AddWithValue("@Id", id);
                using (var reader = cmd.ExecuteReader())
                {
                    if (reader.Read())
                    {
                        return new User
                        {
                            Id = (int)reader["Id"],
                            Email = reader["Email"].ToString(),
                            PasswordHash = reader["PasswordHash"].ToString(),
                            Role = reader["Role"].ToString(),
                            Active = (bool)reader["Active"]
                        };
                    }
                }
            }
        }
        return null;
    }

    // Otros métodos: GetAll, Insert, Update, Delete...
}

// === App_Code/Data/ProductData.cs ===
public class ProductData
{
    public List<Product> GetAll()
    {
        var products = new List<Product>();
        using (var conn = Db.GetConnection())
        {
            conn.Open();
            using (var cmd = new SqlCommand("SELECT * FROM Products", conn))
            {
                using (var reader = cmd.ExecuteReader())
                {
                    while (reader.Read())
                    {
                        products.Add(new Product
                        {
                            Id = (int)reader["Id"],
                            Sku = reader["Sku"].ToString(),
                            Name = reader["Name"].ToString(),
                            Price = (decimal)reader["Price"],
                            Stock = (int)reader["Stock"],
                            Active = (bool)reader["Active"]
                        });
                    }
                }
            }
        }
        return products;
    }

    // Otros métodos: Insert, Update, Delete...
}

// === App_Code/Data/SalesData.cs ===
public class SalesData
{
    public void InsertSale(Sale sale, List<SaleItem> items)
    {
        using (var conn = Db.GetConnection())
        {
            conn.Open();
            using (var transaction = conn.BeginTransaction())
            {
                try
                {
                    // Insertar venta
                    using (var cmd = new SqlCommand("INSERT INTO Sales (CashierUserId, Subtotal, Tax, Total) OUTPUT INSERTED.Id VALUES (@CashierUserId, @Subtotal, @Tax, @Total)", conn, transaction))
                    {
                        cmd.Parameters.AddWithValue("@CashierUserId", sale.CashierUserId);
                        cmd.Parameters.AddWithValue("@Subtotal", sale.Subtotal);
                        cmd.Parameters.AddWithValue("@Tax", sale.Tax);
                        cmd.Parameters.AddWithValue("@Total", sale.Total);
                        sale.Id = (int)cmd.ExecuteScalar();
                    }

                    // Insertar items de venta
                    foreach (var item in items)
                    {
                        using (var cmd = new SqlCommand("INSERT INTO SaleItems (SaleId, ProductId, Quantity, UnitPrice, LineTotal) VALUES (@SaleId, @ProductId, @Quantity, @UnitPrice, @LineTotal)", conn, transaction))
                        {
                            cmd.Parameters.AddWithValue("@SaleId", sale.Id);
                            cmd.Parameters.AddWithValue("@ProductId", item.ProductId);
                            cmd.Parameters.AddWithValue("@Quantity", item.Quantity);
                            cmd.Parameters.AddWithValue("@UnitPrice", item.UnitPrice);
                            cmd.Parameters.AddWithValue("@LineTotal", item.LineTotal);
                            cmd.ExecuteNonQuery();
                        }

                        // Descontar stock
                        using (var cmd = new SqlCommand("UPDATE Products SET Stock = Stock - @Quantity WHERE Id = @ProductId", conn, transaction))
                        {
                            cmd.Parameters.AddWithValue("@Quantity", item.Quantity);
                            cmd.Parameters.AddWithValue("@ProductId", item.ProductId);
                            cmd.ExecuteNonQuery();
                        }
                    }

                    transaction.Commit();
                }
                catch
                {
                    transaction.Rollback();
                    throw; // Manejo de errores en el nivel superior
                }
            }
        }
    }

    // Otros métodos: GetByDateRange...
}
Servicios en App_Code/Services
// === App_Code/Services/AuthService.cs ===
public class AuthService
{
    public bool Login(string email, string password, HttpSessionState session)
    {
        var user = UserData.GetByEmail(email);
        if (user != null && BCrypt.Net.BCrypt.Verify(password, user.PasswordHash))
        {
            session["uid"] = user.Id;
            session["role"] = user.Role;
            return true;
        }
        return false;
    }

    public void Logout(HttpSessionState session)
    {
        session.Clear();
    }
}

// === App_Code/Services/SalesService.cs ===
public class SalesService
{
    public int CreateSale(int cashierUserId, IEnumerable<(int productId, int qty)> items)
    {
        var sale = new Sale
        {
            CashierUserId = cashierUserId,
            Subtotal = items.Sum(i => i.qty * ProductData.GetById(i.productId).Price),
            Tax = items.Sum(i => i.qty * ProductData.GetById(i.productId).Price) * 0.16m,
            Total = items.Sum(i => i.qty * ProductData.GetById(i.productId).Price) * 1.16m
        };

        SalesData.InsertSale(sale, items.Select(i => new SaleItem
        {
            ProductId = i.productId,
            Quantity = i.qty,
            UnitPrice = ProductData.GetById(i.productId).Price,
            LineTotal = i.qty * ProductData.GetById(i.productId).Price
        }).ToList());

        return sale.Id;
    }
}
Páginas (.aspx) con controles ASP.NET y validadores
Login.aspx:
<%@ Page Language="C#" AutoEventWireup="true" CodeBehind="Login.aspx.cs" Inherits="YourNamespace.Login" %>
<!DOCTYPE html>
<html>
<head runat="server">
    <title>Login</title>
</head>
<body>
    <form id="form1" runat="server">
        <div>
            <asp:TextBox ID="txtEmail" runat="server" placeholder="Email" />
            <asp:TextBox ID="txtPassword" runat="server" TextMode="Password" placeholder="Password" />
            <asp:Button ID="btnLogin" runat="server" Text="Iniciar" OnClick="btnLogin_Click" />
            <asp:Label ID="lblMessage" runat="server" ForeColor="Red" />
        </div>
    </form>
</body>
</html>
Default.aspx:
<%@ Page Language="C#" AutoEventWireup="true" CodeBehind="Default.aspx.cs" Inherits="YourNamespace.Default" %>
<!DOCTYPE html>
<html>
<head runat="server">
    <title>Inicio</title>
</head>
<body>
    <form id="form1" runat="server">
        <div>
            <asp:Label ID="lblWelcome" runat="server" />
            <asp:Label ID="lblRole" runat="server" />
        </div>
    </form>
</body>
</html>
Users.aspx:
<%@ Page Language="C#" AutoEventWireup="true" CodeBehind="Users.aspx.cs" Inherits="YourNamespace.Users" %>
<!DOCTYPE html>
<html>
<head runat="server">
    <title>Usuarios</title>
</head>
<body>
    <form id="form1" runat="server">
        <div>
            <asp:GridView ID="gvUsers" runat="server" AutoGenerateColumns="False" OnPageIndexChanging="gvUsers_PageIndexChanging" />
            <asp:FormView ID="fvUser" runat="server" />
            <asp:Label ID="lblMessage" runat="server" ForeColor="Red" />
            <asp:Button ID="btnNew" runat="server" Text="Nuevo" OnClick="btnNew_Click" />
            <asp:Button ID="btnSave" runat="server" Text="Guardar" OnClick="btnSave_Click" />
            <asp:Button ID="btnDelete" runat="server" Text="Eliminar" OnClick="btnDelete_Command" />
        </div>
    </form>
</body>
</html>
Products.aspx:
<%@ Page Language="C#" AutoEventWireup="true" CodeBehind="Products.aspx.cs" Inherits="YourNamespace.Products" %>
<!DOCTYPE html>
<html>
<head runat="server">
    <title>Productos</title>
</head>
<body>
    <form id="form1" runat="server">
        <div>
            <asp:GridView ID="gvProducts" runat="server" AutoGenerateColumns="False" OnPageIndexChanging="gvProducts_PageIndexChanging" />
            <asp:FormView ID="fvProduct" runat="server" />
            <asp:Label ID="lblMessage" runat="server" ForeColor="Red" />
            <asp:Button ID="btnSave" runat="server" Text="Guardar" OnClick="btnSave_Click" />
            <asp:Button ID="btnDelete" runat="server" Text="Eliminar" OnClick="btnDelete_Command" />
        </div>
    </form>
</body>
</html>
CashRegister.aspx:
<%@ Page Language="C#" AutoEventWireup="true" CodeBehind="CashRegister.aspx.cs" Inherits="YourNamespace.CashRegister" %>
<!DOCTYPE html>
<html>
<head runat="server">
    <title>Registro de Caja</title>
</head>
<body>
    <form id="form1" runat="server">
        <div>
            <asp:DropDownList ID="ddlProducts" runat="server" />
            <asp:TextBox ID="txtQty" runat="server" />
            <asp:Button ID="btnAddItem" runat="server" Text="Agregar" OnClick="btnAddItem_Click" />
            <asp:GridView ID="gvCart" runat="server" />
            <asp:Label ID="lblSubtotal" runat="server" />
            <asp:Label ID="lblTax" runat="server" />
            <asp:Label ID="lblTotal" runat="server" />
            <asp:Button ID="btnCheckout" runat="server" Text="Registrar Venta" OnClick="btnCheckout_Click" />
            <asp:Label ID="lblMessage" runat="server" ForeColor="Red" />
        </div>
    </form>
</body>
</html>
SalesReport.aspx:
<%@ Page Language="C#" AutoEventWireup="true" CodeBehind="SalesReport.aspx.cs" Inherits="YourNamespace.SalesReport" %>
<!DOCTYPE html>
<html>
<head runat="server">
    <title>Reporte de Ventas</title>
</head>
<body>
    <form id="form1" runat="server">
        <div>
            <asp:TextBox ID="txtFrom" runat="server" />
            <asp:TextBox ID="txtTo" runat="server" />
            <asp:Button ID="btnFilter" runat="server" Text="Filtrar" OnClick="btnFilter_Click" />
            <asp:GridView ID="gvSales" runat="server" />
            <asp:Label ID="lblTotalGeneral" runat="server" />
            <asp:Label ID="lblMessage" runat="server" ForeColor="Red" />
        </div>
    </form>
</body>
</html>

Necesito que me generes el codigo en C# de cada uno de los botones para guardar la información en la base de datos y mostrarla

  """
})

In [8]:
result2 = model.invoke(result2)
print(result2.content)

A continuación, se presenta el código en C# para los eventos de los botones en las páginas .aspx que has mencionado. Este código incluye la lógica para guardar la información en la base de datos y mostrarla en los controles correspondientes.

### Login.aspx.cs
```csharp
using System;
using System.Web;
using System.Web.UI;

namespace YourNamespace
{
    public partial class Login : Page
    {
        protected void Page_Load(object sender, EventArgs e)
        {
        }

        protected void btnLogin_Click(object sender, EventArgs e)
        {
            var authService = new AuthService();
            if (authService.Login(txtEmail.Text, txtPassword.Text, Session))
            {
                Response.Redirect("Default.aspx");
            }
            else
            {
                lblMessage.Text = "Credenciales inválidas.";
            }
        }
    }
}
```

### Default.aspx.cs
```csharp
using System;
using System.Web;
using System.Web.UI;

namespace YourNamespace
{
   

In [9]:
from pathlib import Path

base = Path().resolve()
outdir = base / "results"          # <— plural
outdir.mkdir(parents=True, exist_ok=True)

outfile = outdir / "Codigo.md"
texto = getattr(result2, "content", None) or str(result2)
outfile.write_text(texto, encoding="utf-8")

print("✅ Guardado en:", outfile.resolve())
print("✅ Guardado en:", outfile.resolve())
print("📏 Tamaño (bytes):", outfile.stat().st_size)
print("🧪 Existe?:", outfile.exists())

✅ Guardado en: /home/runner/work/PuntoVentas/PuntoVentas/results/Codigo.md
✅ Guardado en: /home/runner/work/PuntoVentas/PuntoVentas/results/Codigo.md
📏 Tamaño (bytes): 8631
🧪 Existe?: True


In [10]:
# Asumo que ya tienes: from openai import OpenAI ; client = OpenAI(...)
from openai import OpenAI
import os
import matplotlib.pyplot as plt

# === Carpeta de salida ===
OUTPUT_DIR = "results"
os.makedirs(OUTPUT_DIR, exist_ok=True)
client = OpenAI()
# === PROM (análisis de ventas) ===
prompt = """
Analiza los datos de ventas por tipo de producto y escribe un resumen breve
con conclusiones (categoría con más/menos ventas). Además, indica que se genere
un gráfico de barras con las cantidades por categoría.
Datos:
Bebidas: 120
Snacks: 80
Lácteos: 150
Frutas: 60
Panadería: 90
"""

# === Llamada al modelo (cliente ya configurado arriba) ===
resp = client.chat.completions.create(
    model="gpt-4o",
    messages=[{"role": "user", "content": prompt}],
    temperature=0.2
)

# === Guardar análisis en archivo ===
texto_path = os.path.join(OUTPUT_DIR, "analisis_ventas.txt")
with open(texto_path, "w", encoding="utf-8") as f:
    f.write(resp.choices[0].message.content)

# === Generar y guardar gráfico ===
productos  = ["Bebidas", "Snacks", "Lácteos", "Frutas", "Panadería"]
cantidades = [120, 80, 150, 60, 90]

plt.bar(productos, cantidades)
plt.xlabel("Tipo de Producto")
plt.ylabel("Cantidad Vendida")
plt.title("Ventas por Tipo de Producto")
img_path = os.path.join(OUTPUT_DIR, "ventas_por_producto.png")
plt.savefig(img_path, bbox_inches="tight")
plt.close()

print(f"✅ Guardado:\n- {texto_path}\n- {img_path}")

✅ Guardado:
- results/analisis_ventas.txt
- results/ventas_por_producto.png


In [11]:
# scripts/peritaje_accion.py
# Genera peritaje de código (Markdown + JSON) en results/ analizando archivos dentro de "Codigo/".
# Pensado para ejecutarse en GitHub Actions o localmente.
# No imprime ni usa la API key explícitamente (se toma de la variable de entorno OPENAI_API_KEY).

import os
import glob
from openai import OpenAI

# ---------- Configuración ----------
OUTPUT_DIR = "results"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Rutas a escanear (sensibles a mayúsculas/minúsculas; usa "Codigo/" tal cual está en tu repo)
CANDIDATE_GLOBS = [
    "Codigo/**/*.cs",        # clases y code-behind C#
    "Codigo/**/*.aspx",      # páginas Web Forms
    "Codigo/**/*.aspx.cs",   # code-behind de páginas
    "Codigo/**/*.master",    # master pages
    "Codigo/**/*.config",    # web.config u otros
    "Codigo/*.cs",
    "Codigo/*.aspx",
    "Codigo/*.master",
    "Codigo/*.config",
]

# Límite para no crear prompts gigantes (ajusta si necesitas más contexto)
MAX_TOTAL_CHARS = 20000


# ---------- Utilidades ----------
def lang_for(path: str) -> str:
    ext = os.path.splitext(path)[1].lower()
    if ext == ".cs":
        return "csharp"
    if ext in (".aspx", ".html", ".master"):
        return "html"
    if ext == ".config":
        return "xml"
    return ""


def collect_code_snippets() -> str:
    total = 0
    blocks = []

    # Mostrar archivos detectados (ayuda de depuración)
    found = []
    for pattern in CANDIDATE_GLOBS:
        found += glob.glob(pattern, recursive=True)
    print(f"Archivos detectados para peritaje: {len(found)}")

    for path in found:
        try:
            with open(path, "r", encoding="utf-8", errors="ignore") as f:
                content = f.read()
        except Exception as e:
            print(f"  - No se pudo leer {path}: {e}")
            continue

        if not content.strip():
            continue

        remaining = MAX_TOTAL_CHARS - total
        if remaining <= 0:
            break

        snippet = content[:remaining]
        total += len(snippet)
        lang = lang_for(path)
        fence = f"```{lang}\n{snippet}\n```" if lang else f"```\n{snippet}\n```"
        blocks.append(f"### {path}\n{fence}")

    if not blocks:
        return "_(No se encontraron archivos en 'Codigo/' o se omitieron por tamaño.)_"

    return "\n\n".join(blocks)


def extract_between(s: str, start: str, end: str) -> str:
    a = s.find(start)
    if a == -1:
        return ""
    a += len(start)
    b = s.find(end, a)
    if b == -1:
        return s[a:].strip()
    return s[a:b].strip()


# ---------- Construcción del PROMPT ----------
code_corpus = collect_code_snippets()

PROMPT = f"""
Eres un/a auditor/a senior de código experto/a en seguridad (OWASP), calidad,
rendimiento y mantenibilidad. Analiza el proyecto y entrega **dos salidas**:

1) <<<MARKDOWN>>> ... <<<ENDMARKDOWN>>>  → Reporte en Markdown con:
   - Título: "# Peritaje de código"
   - Resumen ejecutivo (150–250 palabras)
   - Tabla de riesgos (ID, categoría, severidad {{Alta|Media|Baja}}, probabilidad, archivo:línea,
     evidencia, impacto, remediación y parche mínimo en bloque de código)
   - Checklist (OWASP Top 10, sesiones, BCrypt/hash, SQL parametrizado, XSS/CSRF, manejo de errores/logs,
     secretos/keys)
   - Prioridades 30/60/90 días
   - Notas para ASP.NET Web Forms + ADO.NET (SqlParameter, validación server-side, Server.HtmlEncode,
     control de roles, transacciones atómicas, uso de UTC y redondeo monetario)

2) <<<JSON>>> ... <<<ENDJSON>>> → JSON válido con este esquema:
{{
  "summary": "<resumen ejecutivo>",
  "findings": [
    {{
      "id": "R-001",
      "category": "Seguridad|Rendimiento|Calidad|Mantenibilidad|Cumplimiento",
      "severity": "Alta|Media|Baja",
      "probability": "Alta|Media|Baja",
      "file": "ruta/archivo.ext",
      "line": 0,
      "evidence": "fragmento o descripción precisa",
      "impact": "riesgo potencial",
      "remediation": "pasos concretos",
      "patch": "parche mínimo (mismo lenguaje)",
      "references": []
    }}
  ],
  "metrics": {{
    "duplicatedCode": null,
    "complexityHotspots": [],
    "unsafeApis": [],
    "unvalidatedInputs": []
  }}
}}

REGLAS:
- Devuelve ÚNICAMENTE esos dos bloques en ese orden y con esas etiquetas exactas.
- Español claro y accionable.

Contexto:
- Stack: ASP.NET Web Forms + ADO.NET + SQL Server.
- Módulos típicos: Login/Registro, Dashboard, Users, Products, CashRegister, SalesReport.

Código del repositorio (fragmentos de 'Codigo/'):
{code_corpus}
""".strip()


# ---------- Llamada al modelo ----------
# Se asume que OPENAI_API_KEY ya está disponible en el entorno (Actions o local)
client = OpenAI()  # si prefieres: OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

resp = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "Eres un auditor senior de código. Sé preciso, claro y práctico."},
        {"role": "user", "content": PROMPT},
    ],
    temperature=0.2,
)

text = resp.choices[0].message.content or ""


# ---------- Guardado de resultados ----------
md = extract_between(text, "<<<MARKDOWN>>>", "<<<ENDMARKDOWN>>>")
js = extract_between(text, "<<<JSON>>>", "<<<ENDJSON>>>")

md_path = os.path.join(OUTPUT_DIR, "peritaje_codigo.md")
json_path = os.path.join(OUTPUT_DIR, "peritaje_codigo.json")

with open(md_path, "w", encoding="utf-8") as f:
    f.write(md if md.strip() else "# Peritaje de código\n\n_No se pudo extraer el bloque Markdown._")

with open(json_path, "w", encoding="utf-8") as f:
    f.write(js if js.strip() else '{"summary":"No se pudo extraer JSON","findings":[],"metrics":{}}')

print("✅ Resultados guardados:")
print(f" - {md_path}")
print(f" - {json_path}")


Archivos detectados para peritaje: 40


✅ Resultados guardados:
 - results/peritaje_codigo.md
 - results/peritaje_codigo.json
